In [1]:
import pyspark as ps
from pyspark.sql import SQLContext
from pyspark.sql.functions import col
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import CountVectorizer, Tokenizer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_log_error
from sklearn.linear_model import SGDRegressor
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score
import time

spark = (
        ps.sql.SparkSession.builder 
        .master("justin") 
        .appName("chicago") 
        .getOrCreate()
        )

sc = spark.sparkContext

# random.seed(1)

# def sample(p):
#     x, y = random.random(), random.random()
#     return 1 if x*x + y*y < 1 else 0

# count = spark.sparkContext.parallelize(range(0, 10000000)).map(sample) \
#              .reduce(lambda a, b: a + b)

# print("Pi is (very) roughly {}".format(4.0 * count / 10000000))

In [2]:
import pandas as pd
import pymysql


In [3]:
df_sp = spark.read.csv('/home/justin/Downloads/bq-results-20190515-182310-9jmw514pxnfg.csv', header='true',
                       inferSchema='true')

In [4]:
df_pd = df_sp.select('unique_key','arrest','block','community_area','district','latitude','longitude').toPandas()

In [5]:
train = df_pd[df_pd.longitude.isnull()==False]
test = df_pd[df_pd.longitude.isnull()==True]

In [6]:
X_train = train[train.columns.difference(['arrest','longitude'])].block.values
y_train = train['longitude']

In [7]:
vect = TfidfVectorizer()
X_train_vect=vect.fit_transform(X_train)

In [8]:
model = SGDRegressor(loss="squared_loss", penalty='l2', random_state=101, max_iter=5)
params= {'penalty':['none','l2','l1'],
        'alpha':[5e-4,5e-2, .1]}
gs=GridSearchCV(estimator=model,
               param_grid=params,
               scoring='neg_mean_squared_error',
               n_jobs=1,
               cv=5,
               verbose=3)
start=time.time()
gs.fit(X_train_vect, y_train)
end=time.time()
print("Time it took {}".format(end-start))

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] alpha=0.0005, penalty=none ......................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/home/justin/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDRegressor in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


[CV]  alpha=0.0005, penalty=none, score=-32.167207809182436, total=   2.8s
[CV] alpha=0.0005, penalty=none ......................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.8s remaining:    0.0s
/home/justin/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDRegressor in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


[CV]  alpha=0.0005, penalty=none, score=-13.481711735477946, total=   2.4s
[CV] alpha=0.0005, penalty=none ......................................


...

[CV] . alpha=0.1, penalty=l1, score=-10.684420271524422, total=   2.9s
[CV] alpha=0.1, penalty=l1 ...........................................


/home/justin/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDRegressor in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


[CV] .. alpha=0.1, penalty=l1, score=-9.833860093453168, total=   2.9s


[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:  2.0min finished
/home/justin/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDRegressor in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


Time it took 123.41386270523071


In [9]:
model = gs.best_estimator_
pipe = Pipeline([('vect',vect),('model',model)])
y_pred = pipe.predict(test.block.values)

In [12]:
for idx, val in enumerate(test.index):
    df_pd.iloc[val,5] = y_pred[idx] 

In [13]:
train_lat = df_pd[df_pd.latitude.isnull()==False]
test_lat = df_pd[df_pd.latitude.isnull()==True]

In [14]:
X_train_lat = train_lat[train_lat.columns.difference(['arrest','latitude'])].block.values
y_train_lat = train_lat['latitude']

In [15]:
vect_lat = TfidfVectorizer()
X_train_vect_lat=vect_lat.fit_transform(X_train_lat)

In [16]:
model = SGDRegressor(loss="squared_loss", penalty='l2', random_state=101, max_iter=5)
params= {'penalty':['none','l2','l1'],
        'alpha':[5e-4,5e-2, .1]}
gs=GridSearchCV(estimator=model,
               param_grid=params,
               scoring='neg_mean_squared_error',
               n_jobs=1,
               cv=5,
               verbose=3)
start=time.time()
gs.fit(X_train_vect_lat, y_train_lat)
end=time.time()
print("Time it took {}".format(end-start))

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] alpha=0.0005, penalty=none ......................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  alpha=0.0005, penalty=none, score=-7.427539414606433, total=   2.5s
[CV] alpha=0.0005, penalty=none ......................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.6s remaining:    0.0s
/home/justin/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDRegressor in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


[CV]  alpha=0.0005, penalty=none, score=-3.0938211888765483, total=   2.4s
[CV] alpha=0.0005, penalty=none ......................................


...

[CV] . alpha=0.1, penalty=l1, score=-0.5364135631959327, total=   2.8s
[CV] alpha=0.1, penalty=l1 ...........................................


/home/justin/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDRegressor in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


[CV] . alpha=0.1, penalty=l1, score=-0.4528837360798714, total=   2.8s


[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:  2.0min finished
/home/justin/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDRegressor in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


Time it took 123.58108019828796


In [17]:
model1 = gs.best_estimator_
pipe = Pipeline([('vect',vect_lat),('model',model1)])
y_pred = pipe.predict(test_lat.block.values)

In [18]:
for idx, val in enumerate(test_lat.index):
    df_pd.iloc[val,4] = y_pred[idx] 


In [19]:
df_pd.head()

,arrest,block,community_area,district,latitude,longitude
0,True,074XX N OLCOTT AVE,9.0,16.0,42.015614,-87.813912
1,True,051XX S PULASKI RD,62.0,8.0,41.800465,-87.723370
2,False,048XX S KARLOV AVE,57.0,8.0,41.805460,-87.725964
3,True,063XX S NASHVILLE AVE,64.0,8.0,41.777312,-87.786139
4,False,112XX S ELLIS AVE,50.0,5.0,41.690657,-87.605164


In [20]:
df_pd.to_pickle("./chicago.pkl")

In [8]:
unpickled_df = pd.read_pickle("./chicago.pkl")

In [7]:
unpickled_df = pd.concat([df_pd.unique_key, unpickled_df], axis=1, sort=False)

NameError: name 'df_pd' is not defined

In [8]:
unpickled_df = unpickled_df.rename(index=str, columns={'latitude':'Latitude1','longitude':'Longitude1','arrest':'arrest1'})

In [10]:
df_sp2 = spark.createDataFrame(unpickled_df)

In [11]:
df = df_sp.join(df_sp2,['unique_key'],"inner")

In [12]:
df.show(5,False)

+----------+-----------+-----------------------+-----------------------+----+------------+-------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+-----------------------+------------+-------------+-----------------------------+------+-----------------------+--------------+--------+------------+-------------+
|unique_key|case_number|date                   |block                  |iucr|primary_type|description        |location_description|arrest|domestic|beat|district|ward|community_area|fbi_code|x_coordinate|y_coordinate|year|updated_on             |latitude    |longitude    |location                     |arrest|block                  |community_area|district|Latitude1   |Longitude1   |
+----------+-----------+-----------------------+-----------------------+----+------------+-------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+---------

In [56]:
from pyspark.sql.functions import to_timestamp,year, month, dayofmonth, hour,when 
from pyspark.sql.functions import lit

df_sp_dt = df.withColumn('dt',to_timestamp(df_sp.date,'yyyy-MM-dd HH:mm:ss'))\
        .withColumn('Year',year(col("dt")))\
        .withColumn('Month',month(col("dt")))\
        .withColumn('Hour',hour(col("dt")))\
        .withColumn('Night?', when(col('Hour')>=17,1).otherwise(0))\
        .withColumn('Domestic1', when(col('domestic')=='false',0).otherwise(1))\
        .withColumn('Arrest1', when(col('arrest')==True,1).otherwise(0))
        

In [57]:
# df_sp_dt.rdd.saveAsPickleFile("./chicago1.pkl")

# pickleRdd = sc.pickleFile("./chicago1.pkl").collect()
# df2 = spark.createDataFrame(pickleRdd)
df_sp_dt.select('Arrest1')

DataFrame[Arrest1: int]

In [43]:
from pyspark.ml.feature import HashingTF,IDF, Tokenizer


In [44]:
from pyspark.ml.feature import OneHotEncoderEstimator,StringIndexer, VectorAssembler


In [58]:
df_pd2 = df_sp_dt.select('Arrest1','primary_type','fbi_code','month','Night?','Domestic1','Latitude1', 'Longitude1')
df_pd2.printSchema()

root
 |-- Arrest1: integer (nullable = false)
 |-- primary_type: string (nullable = true)
 |-- fbi_code: string (nullable = true)
 |-- month: integer (nullable = true)
 |-- Night?: integer (nullable = false)
 |-- Domestic1: integer (nullable = false)
 |-- Latitude1: double (nullable = true)
 |-- Longitude1: double (nullable = true)



In [59]:
pd.DataFrame(df_pd2.take(5), columns=df_pd2.columns).transpose()

,0,1,2,3,4
Arrest1,1,0,0,1,0
primary_type,HOMICIDE,HOMICIDE,HOMICIDE,HOMICIDE,HOMICIDE
fbi_code,01A,01A,01A,01A,01A
month,6,11,1,10,11
Night?,0,0,0,1,1
Domestic1,0,0,0,0,0
Latitude1,41.7803,41.9327,41.7847,41.9216,41.8843
Longitude1,-87.6642,-87.6928,-87.7205,-87.7328,-87.7606


In [60]:
categorical_columns = ['primary_type','fbi_code','Night?','Domestic1']
stages=[]
for categorical_col in categorical_columns:
    stringIndexer = StringIndexer(inputCol = categorical_col,
                                  outputCol = categorical_col + 'Index')
    encoder = OneHotEncoderEstimator(inputCols = [stringIndexer.getOutputCol()],\
                                     outputCols=[categorical_col + "classVec"])
    stages += [stringIndexer, encoder]

label_stringIdx = StringIndexer(inputCol = 'Arrest1',
                                outputCol = 'label')
stages += [label_stringIdx]

numericalCols = ['Latitude1', 'Longitude1']
assemblerInputs = [c + "classVec" for c in categorical_columns] + numericalCols
assembler = VectorAssembler(inputCols = assemblerInputs,\
                            outputCol = "features")
stages += [assembler]

In [62]:
from pyspark.ml import Pipeline
cols = df_pd2.columns
pipeline = Pipeline(stages = stages)
pipelineModel = pipeline.fit(df_pd2)
df = pipelineModel.transform(df_pd2)
selectedCols = ['label', 'features'] + cols
df = df.select(selectedCols)
df.printSchema()

root
 |-- label: double (nullable = false)
 |-- features: vector (nullable = true)
 |-- Arrest1: integer (nullable = false)
 |-- primary_type: string (nullable = true)
 |-- fbi_code: string (nullable = true)
 |-- month: integer (nullable = true)
 |-- Night?: integer (nullable = false)
 |-- Domestic1: integer (nullable = false)
 |-- Latitude1: double (nullable = true)
 |-- Longitude1: double (nullable = true)



In [63]:
pd.DataFrame(df.take(5), columns = df.columns).transpose()

,0,1,2,3,4
label,1,0,0,1,0
features,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
Arrest1,1,0,0,1,0
primary_type,HOMICIDE,HOMICIDE,HOMICIDE,HOMICIDE,HOMICIDE
fbi_code,01A,01A,01A,01A,01A
month,6,11,1,10,11
Night?,0,0,0,1,1
Domestic1,0,0,0,0,0
Latitude1,41.7803,41.9327,41.7847,41.9216,41.8843
Longitude1,-87.6642,-87.6928,-87.7205,-87.7328,-87.7606


In [65]:
train, test = df.randomSplit([.7,.3], seed = 2019)
print("Training set count {}".format(train.count()))
print("Test set count {}".format(test.count()))

Training set count 1314333
Test set count 563509


In [ ]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(featuresCol = "features", labelCol = 'label', maxIter=10)
lrModel = lr.fit(train)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

beta = np.sort(lrModel.coefficients)
plt.plot(beta)
plt.ylabel('Beta Coefficients')
plt.show()